In [1]:
pip install Augmentor scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Hyperparameters
validation_ratio = 0.10
batch_size = 32
learning_rate = 0.001
epochs = 200
path_to_model ="./checkpoints"

# Augmentation hyperparameters
probability_of_transformation = 0.50

In [3]:
#Load datasets
from tensorflow.keras.datasets import cifar10
import numpy as np

(x_train, digit_y_train), (x_test, digit_y_test) = cifar10.load_data()

In [4]:
#Data conversion, spliting, and scaling
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Scale between 0 and 1
x_train = x_train/255.
x_test = x_test/255.


#Convert labels to one hot encoding
y_train = to_categorical(digit_y_train.flatten(),10)
y_test = to_categorical(digit_y_test.flatten(),10)

# Split data training into sub train and validation
partial_x_train, x_validation, partial_y_train, y_validation = train_test_split(
    x_train, 
    y_train, 
    test_size=validation_ratio, 
    shuffle=True
)

In [5]:
# Data augmentation pipelines
import Augmentor
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augmentor_keras_prprocessing_adaptor(pipeline=None):
    '''
    An adpator to solves scaling conversion problem between augmentor library and ImageDataGenerator
    '''
    def __internalOperation(image):
        if pipeline is not None:
            image = pipeline.keras_preprocess_func()(image)
            image = list(image.getdata())
            image = np.array(image)
            image = image.reshape(( int(np.sqrt(image.shape[0])),int(np.sqrt(image.shape[0])),image.shape[1]))
            return image.astype(np.float) / 255.
        return image
    return __internalOperation


# Extra pipeline for augmentation provided by augmentor
pipeline = Augmentor.Pipeline()
pipeline.flip_left_right(probability_of_transformation)
pipeline.rotate(1,12.5,12.5)
pipeline.shear(1,12.5,12.5)
pipeline.random_brightness(1,0.50,1.50)
pipeline.random_color(1,0.50,1.50)
pipeline.random_contrast(1,0.50,1.50)
pipeline.random_erasing(1,0.5)
    
# Merge pipeline into keras built-in image augmentation
training_data_generator = ImageDataGenerator(featurewise_center= True,
                                             width_shift_range=0.1, 
                                             height_shift_range=0.1, 
                                             preprocessing_function=augmentor_keras_prprocessing_adaptor(pipeline))
validation_data_generator = ImageDataGenerator(featurewise_center= True)
testing_data_generator = ImageDataGenerator(featurewise_center= True)

#Mean normalize all data based on entire training set
training_data_generator.fit(x_train)
validation_data_generator.fit(x_train)
testing_data_generator.fit(x_train)


In [6]:

# #Fully Connected Neural Network
# from keras import models, layers, optimizers, callbacks
# import numpy as np

# fc_model = models.Sequential()
# fc_model.add(layers.InputLayer(input_shape=x_train.shape[1:]))
# fc_model.add(layers.Flatten())

# fc_model.add(layers.Dense(2500, activation="relu"))
# fc_model.add(layers.Dense(2000, activation="relu"))
# fc_model.add(layers.Dense(1500, activation="relu"))
# fc_model.add(layers.Dense(1000, activation="relu"))
# fc_model.add(layers.Dense(500, activation="relu"))

# fc_model.add(layers.Dense(10, activation="softmax"))

# fc_model.compile(
#     optimizer=optimizers.Adam(lr=learning_rate),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

# print(fc_model.summary())

# fc_model.fit_generator(
#     training_data_generator.flow(partial_x_train,partial_y_train,batch_size),
#     validation_data = validation_data_generator.flow(x_validation,y_validation,batch_size),
#     steps_per_epoch = np.ceil(len(partial_x_train)/batch_size),
#     validation_steps = np.ceil(len(x_validation)/batch_size),
#     initial_epoch = 0,
#     epochs=epochs,
# )

# result = fc_model.evaluate_generator(
#     generator = testing_data_generator.flow(x_test,y_test,batch_size),
#     steps = np.ceil(len(x_test)/batch_size)
# )
# print("Testing loss = ", result[0])
# print("Testing accuracy = ", result[1])

In [7]:
from tensorflow.keras import callbacks
import numpy as np
import os
import pathlib

#Helper functions
def get_learning_rate (epoch, lr):
    '''
    Hardset learning rate to be use with learningRateScheduler
    '''
    if epoch > 400:
        return 0.0000001
    elif epoch > 300:
        return 0.000001
    elif epoch > 200:
        return 0.00001
    elif epoch > 100:
        return 0.0001
    return 0.001

class ParameterLogger(callbacks.Callback):
    def __init__(self, epochs_file_path, period):
      self.epochs_file_path = epochs_file_path
      self.period = period
      self.counter = 0 

    def on_epoch_end(self, epoch, logs=None):
        self.counter = self.counter + 1
        if( self.counter == self.period):
          self.counter=0
          with open(self.epochs_file_path, 'w') as f:
            k = epoch+1
            f.write(str(k))

In [8]:
from tensorflow.keras import models, layers, optimizers
import numpy as np
import os
import pathlib

#Build new networks or load existiing one

def build_resnet(filepath):
    '''
    Load existing resnet or create a new one
    '''
    #Create folders for storing weights
    pathlib.Path(os.path.dirname(filepath)).mkdir(parents=True, exist_ok=True)

    #Attempt to load weights checkpoint if it existed
    models_file_path = filepath+".h5"
    epochs_file_path = filepath+".txt"

    model = Resnet()
    epochs = 0

    if(os.path.exists(models_file_path)):
        print('Checkpoint detected.')
        try:
            model.load_weights(models_file_path)
            with open(epochs_file_path, 'r') as f:
              epochs = int(f.read())

            print("Load weights from disks")
        except Exception as e:
            print("Fail to load weights due to wrong models architecture ")

    return model, epochs


def ResnetLayers(previous_layers, filters=128, kernel_size=(3,3), padding='same', strides=1 , activation='relu'):
    #First Conv2D layer
    main_path = layers.BatchNormalization()(previous_layers)
    main_path = layers.Activation(activation)(main_path)
    main_path = layers.Conv2D(filters=filters, kernel_size=kernel_size, padding = padding, strides = strides, kernel_initializer='he_normal')(main_path)
    
    #Second Conv2D layer
    main_path = layers.BatchNormalization()(main_path) 
    main_path = layers.Activation(activation)(main_path)
    main_path = layers.Conv2D(filters=filters, kernel_size=kernel_size, padding = padding, kernel_initializer='he_normal')(main_path)
    

    #Check if inputs have the right shape to perform addition
    if main_path.shape[1:] == previous_layers.shape[1:]:
        shortcut_path = previous_layers
    else:
        shortcut_path = layers.Conv2D(filters=filters, kernel_size=(1,1), padding = padding, strides = strides, activation= activation, kernel_initializer='he_normal')(previous_layers)

    #Complete the second Conv2D layer
    main_path = layers.Add()([main_path, shortcut_path])
    return main_path


def Resnet():
    inputs = layers.Input(shape= x_train.shape[1:])

    #First layers
    outputs = layers.Conv2D(filters=16, kernel_size=(3,3), padding = 'same', strides = 1, kernel_initializer='he_normal')(inputs)
    outputs = layers.BatchNormalization()(outputs)
    outputs = layers.Activation('relu')(outputs)

    #Stacks 1
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=16, kernel_size=(3,3), padding='same', strides=1 , activation='relu')

    #Stacks 2
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=2 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=32, kernel_size=(3,3), padding='same', strides=1 , activation='relu')


    #Stacks 3
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=2 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')
    outputs = ResnetLayers(outputs,  filters=64, kernel_size=(3,3), padding='same', strides=1 , activation='relu')


    outputs = layers.AveragePooling2D(pool_size=8)(outputs)
    outputs = layers.Flatten()(outputs)
    outputs = layers.Dense(10, activation="softmax", kernel_initializer='he_normal')(outputs)

    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=optimizers.Adam(lr=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [9]:
#Convolutional Neural Network
from tensorflow.keras import models, layers, optimizers, callbacks
import numpy as np
import os
import pathlib

#Form  model path 
filespath = os.path.join(path_to_model+"/cnn_model")
        
#Some callbacks
modelCheckpoint = callbacks.ModelCheckpoint(filespath +".h5",save_best_only=True,save_weights_only=True, period=5)
parameterLogger = ParameterLogger(filespath +".txt",  period=5)
learningRateScheduler = callbacks.LearningRateScheduler(get_learning_rate, verbose = 1)
learningReducer = callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=25, min_lr=1e-7, verbose=1)

#Load saved weights
cnn_model, initial_epoch = build_resnet(filespath)

#Print model
print(cnn_model.summary())

#Start training
cnn_model.fit_generator(
    training_data_generator.flow(partial_x_train,partial_y_train,batch_size),
    validation_data = validation_data_generator.flow(x_test,y_test,batch_size),
    steps_per_epoch = np.ceil(len(partial_x_train)/batch_size),
    validation_steps = np.ceil(len(x_test)/batch_size),
    initial_epoch = initial_epoch,
    epochs=epochs,
    callbacks=[learningRateScheduler, modelCheckpoint, parameterLogger]
)

#Evaluate test set
result = cnn_model.evaluate_generator(
    generator = testing_data_generator.flow(x_test,y_test,batch_size),
    steps = np.ceil(len(x_test)/batch_size)
)

#Print results
print("Testing loss = ", result[0])
print("Testing accuracy = ", result[1])

______________________________
conv2d_49 (Conv2D)              (None, 8, 8, 64)     36928       activation_47[0][0]              
__________________________________________________________________________________________________
batch_normalization_48 (BatchNo (None, 8, 8, 64)     256         conv2d_49[0][0]                  
__________________________________________________________________________________________________
activation_48 (Activation)      (None, 8, 8, 64)     0           batch_normalization_48[0][0]     
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 8, 8, 64)     36928       activation_48[0][0]              
__________________________________________________________________________________________________
add_23 (Add)                    (None, 8, 8, 64)     0           conv2d_50[0][0]                  
                                                                 add_22[0][0] 

KeyboardInterrupt: 

In [0]:
#Visualization Hyperparameters

#Color options
colors= ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds']

#Image to apply features map onto
img = x_train[0]

#Maximum numbers of images per layers for each visualization
image_cap = 14

In [0]:
#Visualize Filters

#Imports
import matplotlib.pyplot as plt
import numpy as np

#Helper functions to convert between 1d coordinate and 2d coordinate
def index_to_2d_index (v, width):
    column = v % width
    row = v / width 
    return int(row), int(column)

def two_d_index_to_index(row, column, width):
    return column + (row * width)


#Loop throughts all layers and identify conv2d layers
for layer in cnn_model.layers:
    if 'conv' in layer.name:
        
        #Set internal counter for maximum images per layers
        cap_per_layers = image_cap
        
        #Extract weights and bias
        weights, bias = layer.get_weights()
        kernel_height, kernel_width, num_channels, num_kernels = weights.shape
        
        #Determine a minimum numbers of images slots requires per layers
        width = 7
        height = min(int(np.ceil(num_channels*num_kernels/width)),int(np.ceil(cap_per_layers/width)))
        
        #Set image settings
        fig, axs = plt.subplots(height, width, figsize=(15,2.5*height), squeeze = False)
        fig.suptitle(layer.name, fontsize=20)
        fig.subplots_adjust(top=(height/80)*0.03+0.95)
        
        color_index = np.random.randint(len(colors))
        
        #Start drawing
        for j in range(num_kernels):
            for i in range(num_channels):
                
                #Determines which index to draw at
                index = i + j * num_channels
                
                #Convert to images slots 2d coordinate
                row,column = index_to_2d_index(index,width)
                
                #Draw images
                axs[row,column].imshow(weights[:,:,i,j], cmap=colors[color_index])
                axs[row,column].set_axis_off()
                
                #Update internal image counter
                cap_per_layers = cap_per_layers -1 
                if cap_per_layers<=0:
                    break
            if cap_per_layers<=0:
                    break
        
        #Remove extra image slots that weren't use
        for k in range(1+i + j * num_channels, width*height):
            row,column = index_to_2d_index(k,width)
            fig.delaxes(axs[row][column])
plt.show()


         

In [0]:
#Visualize Features

#Imports
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Model

#Helper functions to convert between 1d coordinate and 2d coordinate
def index_to_2d_index (v, width):
    column = v % width
    row = v / width 
    return int(row), int(column)

def two_d_index_to_index(row, column, width):
    return column + (row * width)

#Extract information from orignal models
names=[]
outputs = []
for layer in cnn_model.layers:
    if  'conv' in layer.name:
        names.append(layer.name)
        outputs.append(layer.output)

#Create new model with multiple outputs        
feature_maps_model = Model(inputs = cnn_model.inputs, outputs = outputs) 

#Use new model to predict an images
result = feature_maps_model.predict(np.array([img]))

if len(outputs) ==1:
    result = [result]

#Loop through output from each convolution layer
for index in range(len(outputs)):
    
    #Set internal counter for maximum images per layers
    cap_per_layers = image_cap
    
    #Reshaping output and extract information
    result[index] = np.squeeze(result[index],axis = 0)

    output_width, output_height, num_kernels = result[index].shape
    
    #Determine a minimum numbers of images slots requires per layers
    width = 7
    height = min(int(np.ceil(num_kernels/width)),int(np.ceil(cap_per_layers/width)))
    
    #Set image settings
    fig, axs = plt.subplots(height, width, figsize=(15,2.5*height), squeeze = False)
    fig.suptitle(names[index], fontsize=20)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=(height/80)*0.03+0.95, wspace=None, hspace=None)
    
    color_index = np.random.randint(len(colors))
    
    #Start drawing
    for i in range(num_kernels):
        
        #Convert to images slots 2d coordinate
        row,column = index_to_2d_index(i,width) 
        
        #Draw images
        axs[row,column].imshow(result[index][:,:,i], cmap=colors[color_index])
        axs[row,column].set_axis_off()
        
        #Update counter
        cap_per_layers = cap_per_layers - 1
        if cap_per_layers<=0:
            break
    #Remove extra image slots that weren't use
    for k in range(i+1, width*height):
        row,column = index_to_2d_index(k,width)
        fig.delaxes(axs[row][column])

plt.show()